In [1]:


// a few dependencies for the notebook:
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
import $ivy.`com.lihaoyi::scalatags:0.12.0`

import better.files._

// load historia code

val jarpath = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
interp.load.cp(os.Path(jarpath))

import $ivy.$                                         

import $ivy.$                             

import $ivy.$                              


import better.files._

// load historia code


jarpath: String = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"

In [2]:



import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull}
import edu.colorado.plv.bounder.lifestate.LifeState

import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil} // Historia utilities
import upickle.default.read
import upickle.default.write
import edu.colorado.plv.bounder.symbolicexecutor.state.DisallowedCallin
import edu.colorado.plv.bounder.lifestate.SAsyncTask
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.{AllMatchers,SJavaThreading,FragmentGetActivityNullSpec, LifeState, LifecycleSpec, RxJavaSpec, SAsyncTask, SDialog, SpecSignatures, SpecSpace, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSTrue, Signature}
import edu.colorado.plv.bounder.symbolicexecutor.PreciseApproxMode

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerate the Historia JAR

def runHistoriaWithSpec(configPath:File, printThenDone:Boolean = false):Driver.LocResult = {
    val javaMemLimit=8 // Gb Note that this only limits JVM not JNI which can go significantly higher
    val historiaJar = jarpath
    val apkRootDir = "/Users/shawnmeier/Documents/data/historia_generalizability"
    val outDir = configPath.parent.toString
    val config = read[RunConfig](configPath.contentAsString)
    val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
    val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m verify -c ${configPath} -b ${apkRootDir} -u ${outDir} -o MEM --debug"
    
    if(printThenDone){
        println(s"cmd ${cmd}")
        ???  // Flag to get cmd to run in ide
    }else{
        //BounderUtil.runCmdStdout(cmd, Some("/Users/shawnmeier/software/z3/build"))
        BounderUtil.runCmdFileOut(cmd, configPath.parent)

        read[Driver.LocResult]((outSubdir / "result_0.txt").contentAsString)
    }
}

def runAndPrint(configPath:File, allSpecs:Iterable[LSSpec], printThenDone:Boolean = false):Unit = {
    val res = runHistoriaWithSpec(configPath,printThenDone)
    println("withist history")
    println(res.witnessExplanation)   
    println("specified messages")
    val msgSigs = allSpecs.flatMap(spec =>
                Set(spec.target) ++ spec.pred.allMsg).map(msg => msg.identitySignature)
    println(msgSigs)
    println(msgSigs.size)
}


import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull}

import edu.colorado.plv.bounder.lifestate.LifeState


import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil} // Historia utilities

import upickle.default.read

import upickle.default.write

import edu.colorado.plv.bounder.symbolicexecutor.state.DisallowedCallin

import edu.colorado.plv.bounder.lifestate.SAsyncTask

import edu.colorado.plv.bounder.PickleSpec

import edu.colorado.plv.bounder.lifestate.{AllMatchers,SJavaThreading,FragmentGetActivityNullSpec, LifeState, LifecycleSpec, RxJavaSpec, SAsyncTask, SDialog, SpecSignatures, SpecSpace, ViewSpec}

import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSTrue, Signature}

import edu.colorado.plv.bounder.symbolicexecutor.PreciseApproxMode

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerat

Row 2 ap ex
-----------


In [ ]:





def run2():Unit = {
    val row2Specs = Set[LSSpec](
        ViewSpec.clickWhileNotDisabled,
        LifecycleSpec.Activity_createdOnlyFirst
      )



    val methodSignature = LifeState.Signature("com.example.row2antennapodexecute.RemoverActivity",
              "void onClick(android.view.View)")


    val initialQuery = DisallowedCallin(
              methodSignature.base,
              methodSignature.methodSignature,
              SAsyncTask.disallowDoubleExecute)
    // val initialQuery = ReceiverNonNull(
    //         methodSignature,
    //         34,
    //         None
    //     )

    val inputApk = "/Users/shawnmeier/Documents/source/historia/Historia/notebooks/Row2_Antennapod_execute_bug/app/build/outputs/apk/debug/app-debug.apk"
    val outputDir = File("/Users/shawnmeier/Documents/source/historia/Historia/notebooks/Row2_Antennapod_execute_bug/WitStoriaOut")
    val cfg =  RunConfig(apkPath = inputApk.toString, 
              outFolder = Some(outputDir.toString),
              initialQuery = List(initialQuery), truncateOut=false, 
                         specSet = PickleSpec(row2Specs, Set(SAsyncTask.disallowDoubleExecute), Set(ViewSpec.fv1)),
                         approxMode = PreciseApproxMode(true)
                        )
    
    val allSpecs = (row2Specs ++ Set[LSSpec](SAsyncTask.disallowDoubleExecute))

    val cfgPath = (outputDir / "cfg.json")
    cfgPath.overwrite(write(cfg))
    

    // val res = runHistoriaWithSpec(cfgPath)
    // println("withist history")
    // println(res.witnessExplanation)
    

    runAndPrint(cfgPath, allSpecs)
}

run2()

Row 1 
-----

Next we create the `RunConfig` that specifies things like the APK, the output folder, the initial query, and the CBCFTL.  This is all written to a `.json` file used later.

In [ ]:
//TODO
val methodSignature = LifeState.Signature("com.example.row1antennapodrxjava.ui.main.PlayerFragment",
          "void call(java.lang.Object)")

val initialQuery = ReceiverNonNull(
        methodSignature,
        63, // line number in source code file
        None //Some(".*toString.*") // regular expression matching receiver (in case multiple dereferences on one line)
    )

val inputApk = "/Users/shawnmeier/Documents/source/historia/Historia/notebooks/AntennapodPlayerFragment_fix/app/build/outputs/apk/debug/app-debug.apk"
val outputDir = File("/Users/shawnmeier/Documents/source/historia/Historia/notebooks/ReachOut")
val cfg =  RunConfig(apkPath = inputApk.toString, 
          outFolder = Some(outputDir.toString),
          initialQuery = List(initialQuery), truncateOut=false)

val cfgPath = (outputDir / "cfg.json")
cfgPath.overwrite(write(cfg))  

Now we can call Historia.

In [ ]:
runHistoriaWithSpec(cfgPath)

Scalability Experiment
======================

Connectbot GeneratePubkeyActivity$5 dialog dismiss

In [4]:
// The method signature unambiguously identifies the method in the application
def runCBGenPub(){
    val methodSignature = LifeState.Signature(
        "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
              "void call(java.lang.Object)")


    val initialQuery = DisallowedCallin(className = "org.connectbot.GeneratePubkeyActivity$5",
                                        methodName = "void run()",
                                        s=SDialog.disallowDismiss)


    val inputApk = 
    "/home/reach_24_data/connectbot/app/build/outputs/apk/oss/debug/app-oss-debug.apk"
    val outputDir = File("/home/notebooks/reachExpLarge/Connectbot/GeneratePubkeyActivity_5_run")
    // val activityMatchers = AllMatchers.iContaining("Activity")
    // println(activityMatchers)
    val cfg =  RunConfig(apkPath = inputApk.toString, 
              outFolder = Some(outputDir.toString),
              initialQuery = List(initialQuery), truncateOut=false,
                        specSet = PickleSpec(Set(ViewSpec.onTouchWhileActive), 
                                             Set(SDialog.disallowDismiss), 
                                             Set(SpecSignatures.Activity_onCreate_entry,
                                                 SJavaThreading.runnableI,
                                             SpecSignatures.Activity_findView_exit)),
                        componentFilter = 
                         Some(Seq("org.connectbot.GeneratePubkeyActivity.*",
                                  "org.connectbot.util.EntropyView.*", 
                                  "org.connectbot.util.PubkeyDatabase.*")),
                        approxMode = PreciseApproxMode(true))


    val cfgPath = (outputDir / "cfg.json")
    cfgPath.overwrite(write(cfg))
    
    val allSpecs = Set(SDialog.disallowDismiss)

    runAndPrint(cfgPath, allSpecs,false)
}
runCBGenPub()

Not mac
withist history
List(Future trace:
 TNew(p-0,{1})
CIExit show(( _T_,_T_ )
TNew(p-3,{2})
CBEnter run()( _T_,@2 )
TNew(p-2,{3})
CBEnter run()( _T_,@1 )
CIEnter void dismiss()( _T_,@5 )
)
specified messages
Set(I_CIEnter_Dialogdismiss, I_CIExit_Dialogshow, I_CBEnter_ActivityonResume, I_CBExit_ActivityonPause)
4


defined function runCBGenPub

In [4]:
// measure for how "useful" this trace is?
// show app is fixed after? but doesn't show we used trace
// 1. can I generate witness with wistoria
//   [ ] come up with multiple witnesses prefereably from "in the wild" issues
//   [ ] 
// 2. can I use trace to witness crash?
// 3. can I fix the app (hopefully with trace help?)
// 4. could we get response to trace that drives interaction? Even without crash
